<a href="https://colab.research.google.com/github/Choihankyul/Enterprise_Demand_Fluctuation_analysis/blob/main/final_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from matplotlib import rc
import matplotlib.pyplot as plt # 한글
rc('font',family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False # 마이너스 부호
import os
import pandas as pd
import seaborn as sns
import numpy as np
from IPython.core.display import HTML
import time
import re

In [ ]:
path = 'dataset' 
file_list = os.listdir(path)

ldata = {}
keys = []
for file in file_list:
    data = pd.read_csv(f'dataset/{file}',encoding='cp949')
    file_name = file.replace('.txt','')
    ldata[file_name]=data
    keys.append(file_name)

def 반기(year,quarter):
    if (year==2014) & (quarter in [1,2]):
        return 1
    elif (year==2014) & (quarter in [3,4]):
        return 2
    elif (year==2015) & (quarter in [1,2]):
        return 3
    else:
        return 4
    
ldata['purprd']['구매일자'] = pd.to_datetime(ldata['purprd']['구매일자'],format='%Y%m%d')
ldata['purprd']['year'] = ldata['purprd']['구매일자'].dt.year
ldata['purprd']['quarter'] = ldata['purprd']['구매일자'].dt.quarter
ldata['purprd']['half'] = ldata['purprd'].apply(lambda x: 반기(x['year'],x['quarter']),axis=1)

productcat = pd.read_excel('상품분류4차.xlsx')
ldata['purprd'] = ldata['purprd'].merge(productcat[['소분류코드','분석대분류명','분류1']],on='소분류코드')
ldata['purprd'] = ldata['purprd'].merge(ldata['prodcat'][['소분류코드','중분류명','소분류명']], on='소분류코드')
ldata['purprd'].rename(columns={'분석대분류명':'대분류명'},inplace=True)
ldata['purprd'].drop(['대분류코드','중분류코드','점포코드','구매시간']
                     ,axis=1,inplace=True)
ldata['purprd'] = ldata['purprd'][ldata['purprd'].제휴사!="D"]
ldata['cust']['거주지역'].fillna(0,inplace=True)

In [ ]:
li = []
for i in ldata['purprd']['대분류명'].unique():
    cat1_mean = round(ldata['purprd'].query(f"대분류명=='{i}'")['구매금액'].mean())
    li.append([i,cat1_mean])    
대분류평균금액 = pd.DataFrame(li,columns=['대분류명','대분류평균금액'])

li = []
for x in ldata['purprd']['대분류명'].unique():
    df = ldata['purprd'].query(f"대분류명=='{x}'")
    for y in df['중분류명'].unique():
        cat2_mean = round(df.query(f'중분류명=="{y}"')['구매금액'].mean())
        li.append([x,y,cat2_mean])

중분류평균금액 = pd.DataFrame(li,columns=['대분류명','중분류명','중분류평균금액'])

df = pd.merge(중분류평균금액,대분류평균금액,on='대분류명')

def cat_amount(cat1, cat1_mean, cat2_mean):
    if cat1_mean*2 <= cat2_mean:
        return "{}고가".format(cat1)
    elif cat1_mean*0.5 <= cat2_mean:
        return "{}중가".format(cat1)
    else:
        return "{}저가".format(cat1)
    
df['상품분류'] = df.apply(lambda x: cat_amount(x['대분류명'],x['대분류평균금액'],x['중분류평균금액']),axis=1)
ldata['purprd'] = ldata['purprd'].merge(df[['대분류명','중분류명','상품분류']],on=['대분류명','중분류명'])

In [ ]:
def original_cust(df):    
    all_cust= pd.pivot_table(df,
                  index='고객번호',
                  columns='half',
                  values='구매금액',
                  aggfunc='sum')
    original_custom_idx =all_cust.dropna().index.tolist()
    original_custom = df.query(f'고객번호 == {original_custom_idx}')
    return original_custom

기존고객 = original_cust(ldata['purprd'])
기존고객idx = 기존고객['고객번호'].unique().tolist()

# <font color=red>__파생변수😀😀__</font>

In [ ]:
# 증감율계산 함수
def rate_variation(f1,f2):
    rate_variation = (f2-f1)/f1
    rate_variation.fillna('부정',inplace=True) # Na : 전 후 둘다 0인 경우
    rate_variation.replace({np.inf:'불능'},inplace=True) # inf : 전반기 0인 경우
    return rate_variation

# 증감폭계산 함수
def variation(f1,f2):
    variation = abs(f2-f1)
    return variation

In [ ]:
# 구매횟수 증감
def p_freq_ratio(n1,n2):
    구매횟수 = pd.pivot_table(기존고객.drop_duplicates('영수증번호'),
             index='고객번호',
             columns='half',
             values='영수증번호',
             aggfunc='count').fillna(0)
    구매횟수증감 = rate_variation(구매횟수[n1],구매횟수[n2])
    return pd.Series(구매횟수증감,name=f'p_freq_ratio')

In [ ]:
# 대분류별 구매금액증감
def cat1_p_amount_ratio(n1,n2):
    대분류구매금액합계 = pd.pivot_table(기존고객,
              index='고객번호',
             columns=['half','대분류명'],
              values='구매금액',
              aggfunc='sum').fillna(0)
    cat1_p_amount = rate_variation(대분류구매금액합계[n1],대분류구매금액합계[n2])
    for colname in cat1_p_amount.columns:
        cat1_p_amount.rename(columns={colname:f'cat1_{colname}_p_amount_ratio'},inplace=True)
    return cat1_p_amount

In [ ]:
# 대분류별 구매물품수증감
def cat1_p_freq_ratio(n1,n2):
    대분류구매물품수합계 = pd.pivot_table(기존고객,
              index='고객번호',
             columns=['half','대분류명'],
              values='구매금액',
              aggfunc='count').fillna(0)
    cat1_p_freq = rate_variation(대분류구매물품수합계[n1],대분류구매물품수합계[n2])
    for colname in cat1_p_freq.columns:
        cat1_p_freq.rename(columns={colname:f'cat1_{colname}_p_freq_ratio'},inplace=True)
    return cat1_p_freq

In [ ]:
# 최대 구매횟수 제휴사(두 반기)
def max_affiliate_p_freq(n1,n2):
    max_구매횟수 = pd.pivot_table(기존고객.drop_duplicates('영수증번호'),
              index='고객번호',
              columns=['half','제휴사'],
              values='영수증번호',
              aggfunc='count')
    max1 = pd.Series(max_구매횟수[n1].transpose().idxmax(),name=n1)
    max2 = pd.Series(max_구매횟수[n2].transpose().idxmax(),name=n2)
    max_affiliate = pd.concat([max1,max2],axis=1)
    max_affiliate['max_affiliate'] = max_affiliate.astype(str).apply('_'.join,axis=1)
    return max_affiliate['max_affiliate']

In [ ]:
# channel 이용유무 : 이용은 1, 이용안했으면 0
def to_binary(x):
    if x>0:
        return 1
    else:
        return 0
def channel():
    channel = pd.pivot_table(ldata['channel'].query(f'고객번호=={기존고객idx}'),
                               index='고객번호',
                               values='이용횟수',
                               aggfunc='sum')
    channel1 = pd.pivot_table(기존고객,
                  index='고객번호',
                  values='제휴사',
                  aggfunc='count').join(channel).fillna(0)[['이용횟수']]    
    return pd.Series(channel1['이용횟수'].apply(lambda x:to_binary(x)),name='channel')

In [ ]:
# 반기별 구매금액증감  
def half_inde(df,*tuple_args):
    buy_sum=pd.pivot_table(df,index='고객번호',
              columns='half',
              values=['구매금액'],
              aggfunc={'구매금액':'sum'})
    for i in tuple_args:        
        buy_sum[f'half_inde'] = (buy_sum[('구매금액',i[1])]-buy_sum[("구매금액",i[0])])/buy_sum[('구매금액',i[0])]
        colname = buy_sum.columns.tolist()[:-1]
        colname.append(f'half_inde_ratio')
        buy_sum.columns = colname    
    buy_sum.drop([('구매금액', 1),
                  ('구매금액', 2),
                  ('구매금액', 3),
                  ('구매금액', 4)],axis=1,inplace=True)           
    return buy_sum

In [ ]:
# 제휴사별 반기별 구매금액증감  
def affiliate_half_inde(df,*tuple_args):
    affiliate_buy_sum=pd.pivot_table(df,index='고객번호',
              columns=['half','제휴사'],
              values=['구매금액'],
              aggfunc={'구매금액':'sum'})       
    for i in tuple_args:
        for j in ['A','B','C']:   
            affiliate_buy_sum['제휴사 구매증감'] = (affiliate_buy_sum[('구매금액',i[1],j)]-affiliate_buy_sum[('구매금액',i[0],j)])/affiliate_buy_sum[('구매금액',i[0],j)]
            colname =affiliate_buy_sum.columns.tolist()[:-1]
            colname.append(f'affiliate_half_inde_{j}_ratio')
            affiliate_buy_sum.columns = colname
    affiliate_buy_sum.fillna('부정',inplace=True) # Na : 전 후 둘다 0인 경우
    affiliate_buy_sum.replace({np.inf:'불능'},inplace=True) # inf : 전반기 0인 경우
    affiliate_buy_sum.drop([('구매금액',1, 'A'),
            ('구매금액',1, 'B'),
            ('구매금액',1, 'C'),
            ('구매금액',2, 'A'),
            ('구매금액',2, 'B'),
            ('구매금액',2, 'C'),
            ('구매금액',3, 'A'),
            ('구매금액',3, 'B'),
            ('구매금액',3, 'C'),
            ('구매금액',4, 'A'),
            ('구매금액',4, 'B'),
            ('구매금액',4, 'C'),],axis=1,inplace=True)
    return affiliate_buy_sum

In [ ]:
# 제휴사별 구매횟수증감 
def buy_count(df,*tuple_args):
    count=pd.pivot_table(df.drop_duplicates('영수증번호'),index='고객번호',
              columns=['half','제휴사'],
              values=['영수증번호'],
              aggfunc={'영수증번호':'count'})    
    for i in tuple_args:
        for j in ['A','B','C']:   
            count['제휴사 구매횟수증감'] = (count[('영수증번호',i[1],j)]-count[('영수증번호',i[0],j)])/count[('영수증번호',i[0],j)]
            colname =count.columns.tolist()[:-1]
            colname.append(f'buy_count_{j}_ratio')
            count.columns = colname
    count.fillna('부정',inplace=True) # Na : 전 후 둘다 0인 경우
    count.replace({np.inf:'불능'},inplace=True) # inf : 전반기 0인 경우
    count.drop([('영수증번호', 1,"A"),
        ('영수증번호', 1,"B"),
        ('영수증번호', 1,"C"),
        ('영수증번호', 2,"A"),
        ('영수증번호', 2,"B"),
        ('영수증번호', 2,"C"),
        ('영수증번호', 3,"A"),
        ('영수증번호', 3,"B"),
        ('영수증번호', 3,"C"),
        ('영수증번호', 4,"A"),
        ('영수증번호', 4,"B"),
        ('영수증번호', 4,"C")],axis=1,inplace=True)  
    return count

In [ ]:
# 일회평균 구매금액 증감
def one_buy_mean_diff(df,*args_tuple):
    buy_sum=pd.pivot_table(df,index='고객번호',
              columns='half',
              values=['구매금액'],
              aggfunc={'구매금액':'sum'})    
    count=pd.pivot_table(df.drop_duplicates('영수증번호'),index='고객번호',
              columns='half',
              values=['영수증번호'],
              aggfunc={'영수증번호':'count'})    
    one_buy_mean_diff=buy_sum.join(count)   
    for i in args_tuple:
        cust_first_mean = one_buy_mean_diff[('구매금액',i[0])]/one_buy_mean_diff[('영수증번호', i[0])]
        cust_second_mean = one_buy_mean_diff[('구매금액',i[1])]/one_buy_mean_diff[('영수증번호', i[1])]
        one_buy_mean_diff['구매평균증감'] = (cust_second_mean - cust_first_mean)/cust_first_mean
        colname = one_buy_mean_diff.columns.tolist()[:-1]
        colname.append(f'one_buy_mean_diff_ratio')
        one_buy_mean_diff.columns = colname
    one_buy_mean_diff.drop([('구매금액', 1),
            ('구매금액', 2),
            ('구매금액', 3),
            ('구매금액', 4),
            ('영수증번호', 1),
            ('영수증번호', 2),
            ('영수증번호', 3),
            ('영수증번호', 4)],axis=1,inplace=True)    
    return one_buy_mean_diff

In [ ]:
# 구매횟수 증감폭
def p_freq_wide(n1,n2):
    구매횟수 = pd.pivot_table(기존고객.drop_duplicates('영수증번호'),
             index='고객번호',
             columns='half',
             values='영수증번호',
             aggfunc='count').fillna(0)
    구매횟수증감 = abs(구매횟수[n2]-구매횟수[n1])
    return pd.Series(구매횟수증감,name=f'p_freq_wide')

In [ ]:
# 대분류별 구매금액증감폭
def cat1_p_amount_wide(n1,n2):
    대분류구매금액합계 = pd.pivot_table(기존고객,
              index='고객번호',
             columns=['half','대분류명'],
              values='구매금액',
              aggfunc='sum').fillna(0)
    cat1_p_amount = abs(대분류구매금액합계[n2]-대분류구매금액합계[n1])
    for colname in cat1_p_amount.columns:
        cat1_p_amount.rename(columns={colname:f'cat1_{colname}_p_amount_wide'},inplace=True)
    return cat1_p_amount

In [ ]:
# 대분류별 구매물품수증감
def cat1_p_freq_wide(n1,n2):
    대분류구매물품수합계 = pd.pivot_table(기존고객,
              index='고객번호',
             columns=['half','대분류명'],
              values='구매금액',
              aggfunc='count').fillna(0)
    cat1_p_freq = abs(대분류구매물품수합계[n2]-대분류구매물품수합계[n1])
    for colname in cat1_p_freq.columns:
        cat1_p_freq.rename(columns={colname:f'cat1_{colname}_p_freq_wide'},inplace=True)
    return cat1_p_freq

In [ ]:
# 반기별 구매금액증감폭
def half_inde_wide(df,*tuple_args):
    buy_sum=pd.pivot_table(df,index='고객번호',
              columns='half',
              values=['구매금액'],
              aggfunc={'구매금액':'sum'})
    for i in tuple_args:        
        buy_sum[f'half_inde{i[1]}_{i[0]}'] = abs(buy_sum[('구매금액',i[1])]-buy_sum[("구매금액",i[0])])
        colname = buy_sum.columns.tolist()[:-1]
        colname.append('half_inde_wide')
        buy_sum.columns = colname    
    buy_sum.drop([('구매금액', 1),
            ('구매금액', 2),
            ('구매금액', 3),
            ('구매금액', 4)],axis=1,inplace=True)        
    return buy_sum

In [ ]:
# 제휴사별 반기별 구매금액증감폭
def affiliate_half_inde_wide(df,*tuple_args):
    affiliate_buy_sum=pd.pivot_table(df,index='고객번호',
              columns=['half','제휴사'],
              values=['구매금액'],
              aggfunc={'구매금액':'sum'})       
    for i in tuple_args:
        for j in ['A','B','C']:   
            affiliate_buy_sum['제휴사 구매증감'] = abs(affiliate_buy_sum[('구매금액',i[1],j)]-affiliate_buy_sum[('구매금액',i[0],j)])
            colname =affiliate_buy_sum.columns.tolist()[:-1]
            colname.append(f'affiliate_{j}_half_inde_wide')
            affiliate_buy_sum.columns = colname
    affiliate_buy_sum.fillna(0,inplace=True)        
    affiliate_buy_sum.drop([('구매금액',1, 'A'),
            ('구매금액',1, 'B'),
            ('구매금액',1, 'C'),
            ('구매금액',2, 'A'),
            ('구매금액',2, 'B'),
            ('구매금액',2, 'C'),
            ('구매금액',3, 'A'),
            ('구매금액',3, 'B'),
            ('구매금액',3, 'C'),
            ('구매금액',4, 'A'),
            ('구매금액',4, 'B'),
            ('구매금액',4, 'C'),],axis=1,inplace=True)
    return affiliate_buy_sum

In [ ]:
# 제휴사별 구매횟수증감폭
def buy_count_wide(df,*tuple_args):
    count=pd.pivot_table(df.drop_duplicates('영수증번호'),index='고객번호',
              columns=['half','제휴사'],
              values=['영수증번호'],
              aggfunc={'영수증번호':'count'})    
    for i in tuple_args:
        for j in ['A','B','C']:   
            count['제휴사 구매횟수증감'] = abs(count[('영수증번호',i[1],j)]-count[('영수증번호',i[0],j)])
            colname =count.columns.tolist()[:-1]
            colname.append(f'buy_count_{j}_wide')
            count.columns = colname
    count.fillna(0,inplace=True)
    count.drop([('영수증번호', 1,"A"),
        ('영수증번호', 1,"B"),
        ('영수증번호', 1,"C"),
        ('영수증번호', 2,"A"),
        ('영수증번호', 2,"B"),
        ('영수증번호', 2,"C"),
        ('영수증번호', 3,"A"),
        ('영수증번호', 3,"B"),
        ('영수증번호', 3,"C"),
        ('영수증번호', 4,"A"),
        ('영수증번호', 4,"B"),
        ('영수증번호', 4,"C")],axis=1,inplace=True)   
    return count

In [ ]:
# 일회평균 구매금액 증감폭
def one_buy_mean_diff_wide(df,*args_tuple):
    buy_sum=pd.pivot_table(df,index='고객번호',
              columns='half',
              values=['구매금액'],
              aggfunc={'구매금액':'sum'})    
    count=pd.pivot_table(df.drop_duplicates('영수증번호'),index='고객번호',
              columns='half',
              values=['영수증번호'],
              aggfunc={'영수증번호':'count'})    
    one_buy_mean_diff=buy_sum.join(count)   
    for i in args_tuple:
        cust_first_mean = one_buy_mean_diff[('구매금액',i[0])]/one_buy_mean_diff[('영수증번호', i[0])]
        cust_second_mean = one_buy_mean_diff[('구매금액',i[1])]/one_buy_mean_diff[('영수증번호', i[1])]
        one_buy_mean_diff['구매평균증감'] = abs(cust_second_mean - cust_first_mean)
        colname = one_buy_mean_diff.columns.tolist()[:-1]
        colname.append(f'one_buy_mean_diff_wide')
        one_buy_mean_diff.columns = colname
    one_buy_mean_diff.drop([('구매금액', 1),
            ('구매금액', 2),
            ('구매금액', 3),
            ('구매금액', 4),
            ('영수증번호', 1),
            ('영수증번호', 2),
            ('영수증번호', 3),
            ('영수증번호', 4)],axis=1,inplace=True)    
    return one_buy_mean_diff

In [ ]:
# 가격대별 구매금액 증감율
def price_ratio(n1,n2):
    def 저중고(x):
        if x[-2:] == '저가':
            return '저가'
        elif x[-2:] == '중가':
            return '중가'
        elif x[-2:] == '고가':
            return '고가'
    기존고객['가격대'] = 기존고객['상품분류'].apply(lambda x: 저중고(x))
    가격대 = pd.pivot_table(기존고객,
                        index='고객번호',
                        columns=['half','가격대'],
                        values='구매금액',
                        aggfunc='sum')
    가격대.fillna(0,inplace=True)
    가격대증감율 = rate_variation(가격대[n1],가격대[n2])    
    for colname in 가격대증감율.columns:
        가격대증감율.rename(columns={colname:f'price_{colname}_ratio'},inplace=True)       
    return 가격대증감율

In [ ]:
# 가격대별 구매금액 증감폭
def price_wide(n1,n2):
    def 저중고(x):
        if x[-2:] == '저가':
            return '저가'
        elif x[-2:] == '중가':
            return '중가'
        elif x[-2:] == '고가':
            return '고가'
    기존고객['가격대'] = 기존고객['상품분류'].apply(lambda x: 저중고(x))
    가격대 = pd.pivot_table(기존고객,
                        index='고객번호',
                        columns=['half','가격대'],
                        values='구매금액',
                        aggfunc='sum')
    가격대.fillna(0,inplace=True)
    가격대증감폭 = variation(가격대[n1],가격대[n2])        
    for colname in 가격대증감폭.columns:
        가격대증감폭.rename(columns={colname:f'price_{colname}_wide'},inplace=True)       
    return 가격대증감폭

In [ ]:
# 대분류/가격대별 구매금액 증감율
def cat1_price_rate(n1,n2):
    대분류가격대 = pd.pivot_table(기존고객,
                        index='고객번호',
                        columns=['half','상품분류'],
                        values='구매금액',
                        aggfunc='sum')
    대분류가격대.fillna(0,inplace=True)
    대분류가격대증감율 = rate_variation(대분류가격대[n1],대분류가격대[n2])
    for colname in 대분류가격대증감율.columns:
        대분류가격대증감율.rename(columns={colname:f'cat1_{colname}_price_ratio'},inplace=True)       
    return 대분류가격대증감율

In [ ]:
# 대분류/가격대별 구매금액 증감폭
def cat1_price_wide(n1,n2):
    대분류가격대 = pd.pivot_table(기존고객,
                        index='고객번호',
                        columns=['half','상품분류'],
                        values='구매금액',
                        aggfunc='sum')
    대분류가격대.fillna(0,inplace=True)
    대분류가격대증감폭 = variation(대분류가격대[n1],대분류가격대[n2])
    for colname in 대분류가격대증감폭.columns:
        대분류가격대증감폭.rename(columns={colname:f'cat1_{colname}_price_wide'},inplace=True)       
    return 대분류가격대증감폭.fillna(0)

In [ ]:
# 대분류별 기여율
def cont_ratio_cat1(n1,n2):
    cat1 = pd.pivot_table(기존고객,
                  index='고객번호',
                  columns=['half','대분류명'],
                  values='구매금액',
                  aggfunc='sum')
    cat1_1 = cat1[n1].fillna(0)
    cat1_1['sum'] = cat1_1.sum(axis=1)
    cat1_2 = cat1[n2].fillna(0)
    cat1_2['sum'] = cat1_2.sum(axis=1)
    cat1기여율 = cat1_2-cat1_1
    for i in 기존고객['대분류명'].unique():
        cat1기여율[i] = cat1기여율[i].div(cat1기여율['sum'])*100
    for colname in cat1기여율.columns:
        cat1기여율.rename(columns={colname:f'cat1_{colname}_cont_ratio'},inplace=True)
    return cat1기여율.drop('cat1_sum_cont_ratio',axis=1)

In [ ]:
# 제휴사별 기여율
def cont_ratio_affiliate(n1,n2):
    제휴사 = pd.pivot_table(기존고객,
                  index='고객번호',
                  columns=['half','제휴사'],
                  values='구매금액',
                  aggfunc='sum')
    제휴사1 = 제휴사[n1].fillna(0)
    제휴사1['sum'] = 제휴사1.sum(axis=1)
    제휴사2 = 제휴사[n2].fillna(0)
    제휴사2['sum'] = 제휴사2.sum(axis=1)
    제휴사기여율 = 제휴사2-제휴사1
    for i in ['A','B','C']:
        제휴사기여율[i] = 제휴사기여율[i].div(제휴사기여율['sum'])*100
    for colname in 제휴사기여율.columns:
        제휴사기여율.rename(columns={colname:f'affiliate_{colname}_cont_ratio'},inplace=True)
    return 제휴사기여율.drop('affiliate_sum_cont_ratio',axis=1).fillna(0)

In [ ]:
def class1_p_amount(n1,n2):
    분류1구매금액합계 = pd.pivot_table(기존고객,
              index='고객번호',
             columns=['half','분류1'],
              values='구매금액',
              aggfunc='sum').fillna(0)
    class1_p_amount = rate_variation(분류1구매금액합계[n1],분류1구매금액합계[n2])
    for colname in class1_p_amount.columns:
        class1_p_amount.rename(columns={colname:f'class1_{colname}_p_amount_ratio'},inplace=True)
    return class1_p_amount

In [ ]:
def class1_p_freq(n1,n2):
    분류1구매물품수합계 = pd.pivot_table(기존고객,
              index='고객번호',
             columns=['half','분류1'],
              values='구매금액',
              aggfunc='count').fillna(0)
    class1_p_freq = rate_variation(분류1구매물품수합계[n1],분류1구매물품수합계[n2])
    for colname in class1_p_freq.columns:
        class1_p_freq.rename(columns={colname:f'class1_{colname}_p_freq_ratio'},inplace=True)
    return class1_p_freq

In [ ]:
def class1_p_amount_wide(n1,n2):
    분류1구매금액합계 = pd.pivot_table(기존고객,
              index='고객번호',
             columns=['half','분류1'],
              values='구매금액',
              aggfunc='sum').fillna(0)
    class1_p_amount = abs(분류1구매금액합계[n2]-분류1구매금액합계[n1])
    for colname in class1_p_amount.columns:
        class1_p_amount.rename(columns={colname:f'class1_{colname}_p_amount_wide'},inplace=True)
    return class1_p_amount

In [ ]:
def class1_p_freq_wide(n1,n2):
    분류1구매물품수합계 = pd.pivot_table(기존고객,
              index='고객번호',
             columns=['half','분류1'],
              values='구매금액',
              aggfunc='count').fillna(0)
    class1_p_freq = abs(분류1구매물품수합계[n2]-분류1구매물품수합계[n1])
    for colname in class1_p_freq.columns:
        class1_p_freq.rename(columns={colname:f'class1_{colname}_p_freq_wide'},inplace=True)
    return class1_p_freq

In [ ]:
def class1_p_freq_net(n1,n2):
    분류1구매물품수합계 = pd.pivot_table(기존고객,
              index='고객번호',
             columns=['half','분류1'],
              values='구매금액',
              aggfunc='count').fillna(0)
    class1_p_freq = 분류1구매물품수합계[n2]-분류1구매물품수합계[n1]
    for colname in class1_p_freq.columns:
        class1_p_freq.rename(columns={colname:f'class1_{colname}_p_freq_net'},inplace=True)
    return class1_p_freq

In [ ]:
def class1_p_amount_net(n1,n2):
    분류1구매금액합계 = pd.pivot_table(기존고객,
              index='고객번호',
             columns=['half','분류1'],
              values='구매금액',
              aggfunc='sum').fillna(0)
    class1_p_amount = 분류1구매금액합계[n2]-분류1구매금액합계[n1]
    for colname in class1_p_amount.columns:
        class1_p_amount.rename(columns={colname:f'class1_{colname}_p_amount_net'},inplace=True)
    return class1_p_amount

In [ ]:
# 식품비중
def food_w(n1,n2):
    p_amount = pd.pivot_table(기존고객,
                  index='고객번호',
                  columns=['half','대분류명'],
                  values='구매금액',
                  aggfunc='sum')
    p_amount_t = p_amount[n1] + p_amount[n2]
    p_amount_t['sum'] = p_amount_t.sum(axis=1)
    p_amount_t['food_w'] = p_amount_t['가공식품'] + p_amount_t['신선식품']
    food_w = p_amount_t['food_w'].div(p_amount_t['sum'],axis=0)*100
    return food_w.fillna(0)

In [ ]:
# 선매품/일상품/전문품 비중
def class1_w(n1,n2):
    class1_amount = pd.pivot_table(기존고객,
                  index='고객번호',
                  columns=['half','분류1'],
                  values='구매금액',
                  aggfunc='sum')
    class1_amount_t = class1_amount[n1] + class1_amount[n2]
    class1_amount_t['sum'] = class1_amount_t.sum(axis=1)
    class1_w = class1_amount_t.drop('sum',axis=1).div(class1_amount_t['sum'],axis=0)*100
    for colname in class1_w.columns:
        class1_w.rename(columns={colname:f'class1_w_{colname}'},inplace=True)
    return  class1_w.fillna(0)

In [ ]:
# 종속변수
def label(df,half1,half2):
    sale_different = pd.pivot_table(df,index='고객번호',
                                   columns = 'half',
                                   values = '구매금액',
                                   aggfunc= 'sum')
    전체증감율 = (sum(sale_different[int(f'{half2}')])-sum(sale_different[int(f'{half1}')]))/sum(sale_different[int(f'{half1}')])
    sale_different['label'] = (sale_different[int(f'{half2}')] - sale_different[int(f'{half1}')])/sale_different[int(f'{half1}')]/전체증감율
    def to_label(증감):
        if 증감>=1:
            return 0
        else:
            return 1
    sale_different[f'label'] = sale_different[f'label'].apply(lambda x: to_label(x))
    sale_different = sale_different[[f'label']]
    sale_different.columns = [f'label'] 
    return sale_different

 # <font color=red>__데이터셋😀😀__</font>

In [ ]:
def make_dataset(df,n1,n2,n3):
    dataset = pd.DataFrame(p_freq_ratio(n1,n2)).join([cat1_p_amount_ratio(n1,n2), # 반기별구매횟수증감율, 대분류별구매금액증감율
                                   cat1_p_freq_ratio(n1,n2), # 대분류별 구매물품수증감율
                                   max_affiliate_p_freq(n1,n2), # 최대구매금액제휴사 변동
                                   channel(), # 채널이용유무
                                   half_inde(df,(n1,n2)), # 반기별구매금액증감율
                                   affiliate_half_inde(df,(n1,n2)), # 제휴사별구매금액증감율
                                   buy_count(df,(n1,n2)), # 반기별구매횟수증감율
                                   one_buy_mean_diff(df,(n1,n2)), # 1회평균구매금액증감율
                                                
                                   ldata['cust'].set_index('고객번호'), # 성별, 연령대, 거주지역
                                                
                                   p_freq_wide(n1,n2), # 구매횟수변동폭
                                   cat1_p_amount_wide(n1,n2), # 대분류별구매금액변동폭
                                   cat1_p_freq_wide(n1,n2), # 대분류별구매물품수변동폭
                                   half_inde_wide(df,(n1,n2)), # 반기별구매금액변동폭
                                   affiliate_half_inde_wide(df,(n1,n2)), # 제휴사별구매금액변동폭
                                   buy_count_wide(df,(n1,n2)), # 반기별구매횟수변동폭
                                   one_buy_mean_diff_wide(df,(n1,n2)), # 1회평균구매금액변동폭
                                                
                                   price_ratio(n1,n2), # 가격대구매금액증감율
                                   price_wide(n1,n2), # 가격대구매금액변동폭
                                   cat1_price_rate(n1,n2), # 대분류별가격대구매금액증감율
                                   cat1_price_wide(n1,n2), # 대분류별가격대구매금액변동폭
                                   cont_ratio_cat1(n1,n2), # 대분류별구매금액기여율
                                   cont_ratio_affiliate(n1,n2), # 제휴사별구매금액기여율
                                                
                                   class1_p_amount(n1,n2), # 분류1 구매금액 증감율
                                   class1_p_freq(n1,n2), # 분류1 구매횟수 증감율
                                   class1_p_amount_wide(n1,n2), # 분류1 구매금액 변동폭
                                   class1_p_freq_wide(n1,n2), # 분류1 구매횟수 변동폭
                                   class1_p_freq_net(n1,n2), # 분류1 구매횟수 증감
                                   class1_p_amount_net(n1,n2), # 분류1 구매금액 증감
                                   label(df,n1,n3)]) # 종속변수
    return dataset

dataset1_2 = make_dataset(기존고객,1,2,3)
dataset2_3 = make_dataset(기존고객,2,3,4)

In [ ]:
#### 카테고리 전
dataset1_2.to_excel('dataset_1017/dataset1_2.xlsx')
dataset2_3.to_excel('dataset_1017/dataset2_3.xlsx')

## 범주화

In [ ]:
dataset1_2_cat = dataset1_2.copy()
dataset2_3_cat = dataset2_3.copy()

In [ ]:
dataset1_2_cat.drop(['cat1_기타저가_price_ratio','cat1_기타저가_price_wide','cat1_의약고가_price_wide','cat1_의약저가_price_ratio','cat1_의약중가_price_ratio','cat1_의약고가_price_ratio'],axis=1,inplace=True)
dataset2_3_cat.drop(['cat1_기타저가_price_ratio','cat1_기타저가_price_wide','cat1_의약고가_price_wide','cat1_의약저가_price_ratio','cat1_의약중가_price_ratio','cat1_의약고가_price_ratio'],axis=1,inplace=True)

In [ ]:
# 카테고리 대상 컬럼  

to_cat_columns =list(dataset1_2_cat.columns[dataset1_2_cat.columns.str.contains('ratio')])
to_cat_wide_columns = list(dataset1_2_cat.columns[dataset1_2_cat.columns.str.contains('wide')])
to_cat_net_columns = list(dataset1_2_cat.columns[dataset1_2_cat.columns.str.contains('net')])
le_columns = ['성별','연령대','거주지역','max_affiliate']

In [ ]:
def to_cat(df, col,n=4):
    data = df[col]
    data_float = data[(data!='불능')&(data!='부정')].astype(float)
    if len(data_float) != 0:
        data_minus = data_float[data_float<=0]
        data_minus_qcut = pd.cut(data_minus,n,labels=list(range(1,n+1)))

        data_plus = data_float[(0<data_float)&(data_float<=1)]
        data_plus_qcut = pd.cut(data_plus,n,labels=list(range(1+n,n*2+1)))

        data_float[1<data_float] = 2*n+1
        data_outlier = data_float[1<data_float]

        data_object = data[(data=='불능')|(data=='부정')]
        data_object.replace({'부정':0,'불능':2*n+2},inplace=True)
        return pd.concat([data_minus_qcut,data_plus_qcut,data_outlier,data_object])
    
    else:
        data_object = data[(data=='불능')|(data=='부정')]
        data_object.replace({'부정':0,'불능':1},inplace=True)
        return data_object

for i in to_cat_columns:
    dataset1_2_cat[i] = to_cat(dataset1_2_cat,i)
for i in to_cat_columns:
    dataset2_3_cat[i] = to_cat(dataset2_3_cat,i)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for i in le_columns:
    dataset1_2_cat[i] = le.fit_transform(dataset1_2_cat[i])
for i in le_columns:
    dataset2_3_cat[i] = le.fit_transform(dataset2_3_cat[i])

In [ ]:
def to_cat_wide(df,col,n=4):
    data = df[col]
    q1, q3 = np.percentile(df[col],[25,75])    
    iqr=q3 - q1    
    lower_bound = q1-(iqr * 1.5)
    upper_bound = q3+(iqr * 1.5)

    def deny(x):
        if x >upper_bound:
            return "up"
        elif x < lower_bound:
            return "lo"
        else:
            return x

    data = df[col].apply(deny)
    data_float = data[(data!='lo')&(data!='up')].astype(float)

    if len(data_float) != 0:
        data_float_cut = pd.cut(data_float,n,labels=list(range(1,n+1)))

        data_object = data[(data=='lo')|(data=='up')]
        data_object.replace({'up':n+1,'lo':0},inplace=True)

        return pd.concat([data_float_cut,data_object])
        
    else:
        return df[col].fillna(0)

for i in to_cat_wide_columns:
    dataset1_2_cat[i] = to_cat_wide(dataset1_2_cat,i)
for i in to_cat_wide_columns:
    dataset2_3_cat[i] = to_cat_wide(dataset2_3_cat,i)

In [ ]:
def to_cat_net(df,col,n=4):
    data = df[col]
    q1, q3 = np.percentile(df[col],[25,75])    
    iqr=q3 - q1    
    lower_bound = q1-(iqr * 1.5)
    upper_bound = q3+(iqr * 1.5)

    def deny(x):
        if x >upper_bound:
            return "up"
        elif x < lower_bound:
            return "lo"
        else:
            return x

    data = df[col].apply(deny)
    data_float = data[(data!='lo')&(data!='up')].astype(float)

    if len(data_float) != 0:
        data_float_cut = pd.cut(data_float,n+1,labels=list(range(1,n+2)))

        data_object = data[(data=='lo')|(data=='up')]
        data_object.replace({'up':n+2,'lo':0},inplace=True)

        return pd.concat([data_float_cut,data_object])
        
    else:
        return df[col].fillna(0)
    
for i in to_cat_net_columns:
    dataset1_2_cat[i] = to_cat_net(dataset1_2_cat,i)
for i in to_cat_net_columns:
    dataset2_3_cat[i] = to_cat_net(dataset2_3_cat,i)

In [ ]:
#### 카테고리 후
dataset1_2_cat.to_excel('dataset_1017/dataset1_2_cat.xlsx')
dataset2_3_cat.to_excel('dataset_1017/dataset2_3_cat.xlsx')

### 모델링

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import sklearn.svm as svm
import sklearn.metrics as mt
from xgboost import plot_importance
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier, plot_importance

In [ ]:
# 3기 예측

X = dataset1_2.astype(float).drop('label',axis=1)
y = dataset1_2['label']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3,random_state = 4)

def metric(y_test, pred):
    print('-----------------------------')
    print('정확도 : ',round(mt.accuracy_score(y_test, pred),2))
    print('정밀도 : ',round(mt.precision_score(y_test, pred),2))
    print('재현율 : ',round(mt.recall_score(y_test, pred),2))
    print('f1 : ',round(mt.f1_score(y_test, pred),2))
    print('\n')
    
dct = DecisionTreeClassifier(criterion = 'entropy',max_depth = 10,random_state=2)
rfc = RandomForestClassifier(n_estimators = 10)
log = LogisticRegression()

rfc.fit(X_train, y_train)
dct.fit(X_train, y_train)
log.fit(X_train, y_train)

dct_pred = dct.predict(X_test)
rfc_pred = rfc.predict(X_test)
log_pred = log.predict(X_test)

xgb_clf = XGBClassifier(n_estimators=300, random_state=156, learning_rate=0.1, max_depth=5) 
xgb_clf.fit(X_train, y_train,
           eval_metric='auc', eval_set=[(X_train,y_train),(X_test,y_test)],verbose=False)
xgb_pred = xgb_clf.predict(X_test)


lgb = LGBMClassifier()
lgb.fit(X_train, y_train)
lgb_pred = lgb.predict(X_test)

print('로지스틱')
metric(y_test, log_pred)
print('랜덤포레스트')
metric(y_test, rfc_pred)
print('디시젼')
metric(y_test, dct_pred)
print('XGB')
metric(y_test, xgb_pred)
print('LightGBM')
metric(y_test, lgb_pred)

In [ ]:
# 1~3기 예측 모델 적용해서 4기 예측

X_test1 = dataset2_3.astype(float).drop('label',axis=1)
y_test1 = dataset2_3['label']

dct_pred1 = dct.predict(X_test1)
rfc_pred1 = rfc.predict(X_test1)
log_pred1 = log.predict(X_test1)
xgb_pred1 = xgb_clf.predict(X_test1)
lgb_pred1 = lgb.predict(X_test1)

print('로지스틱')
metric(y_test1, log_pred1)
print('랜덤포레스트')
metric(y_test1, rfc_pred1)
print('디시젼')
metric(y_test1, dct_pred1)
print('XGB')
metric(y_test1, xgb_pred1)
print('LightGBM')
metric(y_test1, lgb_pred1)

In [ ]:
# 로지스틱 회귀계수로 피처 선택 후 학습 및 예측
import statsmodels.api as sm
summary=sm.Logit(y, X).fit().summary().tables[1].as_html()
pvalues = pd.read_html(summary, header=0, index_col=0)[0]
display(pvalues,pvalues.iloc[:,3][pvalues.iloc[:,3]<0.05])

columns = pvalues.iloc[:,3][pvalues.iloc[:,3]<0.05].index.tolist()

# X = dataset_last1_2_cat1.drop('label',axis=1)
# y = dataset_last1_2_cat1['label']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3,random_state = 4)

def metric(y_test, pred):
    print('-----------------------------')
    print('정확도 : ',round(mt.accuracy_score(y_test, pred),2))
    print('정밀도 : ',round(mt.precision_score(y_test, pred),2))
    print('재현율 : ',round(mt.recall_score(y_test, pred),2))
    print('f1 : ',round(mt.f1_score(y_test, pred),2))
    print('\n')
    
dct = DecisionTreeClassifier(criterion = 'entropy',max_depth = 10,random_state=2)
rfc = RandomForestClassifier(n_estimators = 10)
log = LogisticRegression()

rfc.fit(X_train, y_train)
dct.fit(X_train, y_train)
log.fit(X_train, y_train)

dct_pred = dct.predict(X_test)
rfc_pred = rfc.predict(X_test)
log_pred = log.predict(X_test)

xgb_clf = XGBClassifier(n_estimators=1000, random_state=156, learning_rate=0.01, max_depth=5) 
xgb_clf.fit(X_train, y_train, early_stopping_rounds=100,
           eval_metric='auc', eval_set=[(X_train,y_train),(X_test,y_test)],verbose=False)
xgb_pred = xgb_clf.predict(X_test)

lgb = LGBMClassifier()
lgb.fit(X_train, y_train)
lgb_pred = lgb.predict(X_test)

print('로지스틱')
metric(y_test, log_pred)
print('랜덤포레스트')
metric(y_test, rfc_pred)
print('디시젼')
metric(y_test, dct_pred)
print('XGB')
metric(y_test, xgb_pred)
print('LightGBM')
metric(y_test, lgb_pred)

In [ ]:
# feature importance 확인


from xgboost import XGBClassifier

model = XGBClassifier(random_state=4)
model.fit(X_train, y_train)

ft_importance_values = model.feature_importances_

ft_series = pd.Series(ft_importance_values, index = X_train.columns)
ft_top30 = ft_series.sort_values(ascending=False)[:30]

plt.figure(figsize=(10,10))
plt.title('Feature Importance Top 20')
sns.barplot(x=ft_top30, y=ft_top30.index)
plt.show()

### 감소고객 군집화 및 DecisionTree 시각화

In [ ]:
month = dataset1_2.columns[dataset1_2.columns.str.contains('month')]
dataset1_2.drop(month,axis=1,inplace=True)
month1 = dataset2_3.columns[dataset2_3.columns.str.contains('month')]
dataset2_3.drop(month1,axis=1,inplace=True)

# 4기 감소고객
감소고객 = dataset2_3[dataset2_3['label']==1]

from sklearn.cluster import KMeans
%matplotlib inline

from yellowbrick.cluster import KElbowVisualizer
 
x = 감소고객[['cat1_p_amount_가공식품_ratio',
'cat1_p_amount_기타_ratio',     
'cat1_p_amount_미용품_ratio',      
'cat1_p_amount_스포츠레저_ratio', 
'cat1_p_amount_신선식품_ratio',    
'cat1_p_freq_가공식품_ratio',       
'cat1_p_freq_기타_ratio',         
'cat1_p_freq_신선식품_ratio',       
'cat1_p_freq_일상용품_ratio',       
'max_affiliate',                
'half_inde_ratio',              
'rate_p_freq_wide',             
'cat1_p_amount_인테리어_wide',      
'cat1_p_freq_스포츠레저_wide',       
'cat1_p_freq_신선식품_wide',        
'buy_count_A_wide',             
'price_ratio_고가',               
'price_ratio_저가',              
'price_ratio_중가',               
'price_wide_고가',               
'price_wide_저가',                
'cat1_price_ratio_교육문화고가',      
'cat1_price_ratio_기타고가',        
'cat1_price_ratio_기타중가',        
'cat1_price_ratio_디지털고가',       
'cat1_price_ratio_디지털저가',       
'cat1_price_ratio_디지털중가',       
'cat1_price_ratio_스포츠레저중가',     
'cat1_price_ratio_인테리어고가',      
'cat1_price_ratio_패션잡화중가',      
'cat1_price_wide_가공식품중가',       
'cat1_price_wide_교육문화고가',       
'cat1_price_wide_기타고가',         
'cat1_price_wide_디지털고가',        
'cat1_price_wide_디지털저가',        
'cat1_price_wide_스포츠레저저가',      
'cat1_price_wide_신선식품중가',       
'cat1_price_wide_의약저가',         
'cat1_price_wide_인테리어고가',       
'cat1_price_wide_인테리어중가',       
'cont_ratio_cat1가공식품',          
'cont_ratio_cat1디지털',           
'cont_ratio_affiliateA',        
'class1_p_amount_선매품_ratio',    
'class1_p_amount_선매품_wide',     
'class1_p_freq_편의품_wide',       
'class1_p_freq_편의품_net']]


def elbow(X):
    sse = []

    for i in range(1,11):
        km = KMeans(n_clusters=i,algorithm='auto', random_state=42)
        km.fit(X)
        sse.append(km.inertia_)

    plt.plot(range(1,11), sse, marker='o')
    plt.xlabel('K')
    plt.ylabel('SSE')
    plt.show()

elbow(x)

In [ ]:
def visualize_silhouette(cluster_lists, X_features): 

    from sklearn.datasets import make_blobs
    from sklearn.cluster import KMeans
    from sklearn.metrics import silhouette_samples, silhouette_score

    import matplotlib.pyplot as plt
    import matplotlib.cm as cm
    import math

    # 입력값으로 클러스터링 갯수들을 리스트로 받아서, 각 갯수별로 클러스터링을 적용하고 실루엣 개수를 구함
    n_cols = len(cluster_lists)

    # plt.subplots()으로 리스트에 기재된 클러스터링 수만큼의 sub figures를 가지는 axs 생성 
    fig, axs = plt.subplots(figsize=(4*n_cols, 4), nrows=1, ncols=n_cols)

    # 리스트에 기재된 클러스터링 갯수들을 차례로 iteration 수행하면서 실루엣 개수 시각화
    for ind, n_cluster in enumerate(cluster_lists):

        # KMeans 클러스터링 수행하고, 실루엣 스코어와 개별 데이터의 실루엣 값 계산. 
        clusterer = KMeans(n_clusters = n_cluster, max_iter=500, random_state=0)
        cluster_labels = clusterer.fit_predict(X_features)

        sil_avg = silhouette_score(X_features, cluster_labels)
        sil_values = silhouette_samples(X_features, cluster_labels)

        y_lower = 10
        axs[ind].set_title('Number of Cluster : '+ str(n_cluster)+'\n' \
                          'Silhouette Score :' + str(round(sil_avg,3)) )
        axs[ind].set_xlabel("The silhouette coefficient values")
        axs[ind].set_ylabel("Cluster label")
        axs[ind].set_xlim([-0.1, 1])
        axs[ind].set_ylim([0, len(X_features) + (n_cluster + 1) * 10])
        axs[ind].set_yticks([])  # Clear the yaxis labels / ticks
        axs[ind].set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1])

        # 클러스터링 갯수별로 fill_betweenx( )형태의 막대 그래프 표현. 
        for i in range(n_cluster):
            ith_cluster_sil_values = sil_values[cluster_labels==i]
            ith_cluster_sil_values.sort()

            size_cluster_i = ith_cluster_sil_values.shape[0]
            y_upper = y_lower + size_cluster_i

            color = cm.nipy_spectral(float(i) / n_cluster)
            axs[ind].fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_sil_values, \
                                facecolor=color, edgecolor=color, alpha=0.7)
            axs[ind].text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
            y_lower = y_upper + 10

        axs[ind].axvline(x=sil_avg, color="red", linestyle="--")

        


import numpy as np
# make_blobs 을 통해 clustering 을 위한 4개의 클러스터 중심의 500개 2차원 데이터 셋 생성  
from sklearn.datasets import make_blobs
X, y = make_blobs(n_samples=500, n_features=2, centers=4, cluster_std=1, \
                  center_box=(-10.0, 10.0), shuffle=True, random_state=1)  

# cluster 개수를 2개, 3개, 4개, 5개 일때의 클러스터별 실루엣 계수 평균값을 시각화 
visualize_silhouette([ 2, 3, 4, 5], X)

In [ ]:
kmeans = KMeans(n_clusters=4, init='k-means++', max_iter=300, random_state=0)
kmeans.fit(x)
감소고객['cluster']=kmeans.labels_

In [ ]:
group0 = 감소고객[감소고객['cluster']==0]
group1 = 감소고객[감소고객['cluster']==1]
group2 = 감소고객[감소고객['cluster']==2]
group3 = 감소고객[감소고객['cluster']==3]

In [ ]:
from sklearn.tree import export_graphviz

export_graphviz(dt_clf, out_file='tree.dot',class_names=['0','1','2','3'], feature_names=['cat1_p_amount_가공식품_ratio',
'cat1_p_amount_기타_ratio',     
'cat1_p_amount_미용품_ratio',      
'cat1_p_amount_스포츠레저_ratio', 
'cat1_p_amount_신선식품_ratio',    
'cat1_p_freq_가공식품_ratio',       
'cat1_p_freq_기타_ratio',         
'cat1_p_freq_신선식품_ratio',       
'cat1_p_freq_일상용품_ratio',       
'max_affiliate',                
'half_inde_ratio',              
'rate_p_freq_wide',             
'cat1_p_amount_인테리어_wide',      
'cat1_p_freq_스포츠레저_wide',       
'cat1_p_freq_신선식품_wide',        
'buy_count_A_wide',             
'price_ratio_고가',               
'price_ratio_저가',              
'price_ratio_중가',               
'price_wide_고가',               
'price_wide_저가',                
'cat1_price_ratio_교육문화고가',      
'cat1_price_ratio_기타고가',        
'cat1_price_ratio_기타중가',        
'cat1_price_ratio_디지털고가',       
'cat1_price_ratio_디지털저가',       
'cat1_price_ratio_디지털중가',       
'cat1_price_ratio_스포츠레저중가',     
'cat1_price_ratio_인테리어고가',      
'cat1_price_ratio_패션잡화중가',      
'cat1_price_wide_가공식품중가',       
'cat1_price_wide_교육문화고가',       
'cat1_price_wide_기타고가',         
'cat1_price_wide_디지털고가',        
'cat1_price_wide_디지털저가',        
'cat1_price_wide_스포츠레저저가',      
'cat1_price_wide_신선식품중가',       
'cat1_price_wide_의약저가',         
'cat1_price_wide_인테리어고가',       
'cat1_price_wide_인테리어중가',       
'cont_ratio_cat1가공식품',          
'cont_ratio_cat1디지털',           
'cont_ratio_affiliateA',        
'class1_p_amount_선매품_ratio',    
'class1_p_amount_선매품_wide',     
'class1_p_freq_편의품_wide',       
'class1_p_freq_편의품_net'], impurity=True, filled=True)

In [ ]:
import graphviz

with  open('tree.dot',encoding='UTF8') as f:
    dot_graph = f.read()
graphviz.Source(dot_graph)